In [ ]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
###################
disp=[];nf=[]
for i in range(12001):
    df=pd.read_excel('data/raw_data/12000_displacements/CaseJuly'+str(i+1)+'.xlsx')
    nf.append(df.iloc[0,:7].values.astype('float64'))
    df=df.values[2:,list(range(3,3*7+1,3))]#df=df.values[1:,3]
    disp.append(df)

disp=np.stack(disp,axis=0)
nf=np.stack(nf,axis=0)
print(disp.shape,nf.shape)
disp=disp.astype('float64')
print(np.min(disp),np.max(disp),np.mean(disp),np.std(disp))
print(np.min(nf),np.max(nf),np.mean(nf),np.std(nf))
np.save('data/disp',disp)
np.save('data/nf',nf)

area=pd.read_excel('data/raw_data/12000_displacements/INPUT_12000.xlsx')
area=area.values[:,1:];print(area.shape)#(12000, 120)
np.save('data/data_cablearea',area)

In [8]:
import numpy as np
import pandas as pd
import pickle
import math
import matplotlib.pyplot as plt

node_loc=(pd.read_excel('data/raw_data/node_location.xlsx',header=None).values)
element_cable=(pd.read_excel('data/raw_data/element_cable.xlsx').iloc[:,[1,2,4,5]]).values
element_tower=(pd.read_excel('data/raw_data/element_tower.xlsx').iloc[:,[1,2,4,5]]).values
element_girder=(pd.read_excel('data/raw_data/element_girder.xlsx').iloc[:,[1,2,4,5]]).values
element_crossbeam=(pd.read_excel('data/raw_data/element_crossbeam.xlsx').iloc[:,[1,2,4,5]]).values
element_virtual=(pd.read_excel('data/raw_data/element_virtual.xlsx',header=None)).values
element_virtual=element_virtual-1
elements=np.concatenate((element_cable,element_tower,element_girder,element_crossbeam),axis=0)
elements[:,[0,1]]=elements[:,[0,1]]-1
print(np.min(elements[:,[0,1]]),np.max(elements[:,[0,1]]))
#adj_mx
num_node=len(node_loc);print('num node: ',num_node)
adj_mx = np.zeros((num_node, num_node), dtype=np.float32)
adj_mx[elements[:,0],elements[:,1]]=1
adj_mx[element_virtual[:,0],element_virtual[:,1]]=1
adj_mx = np.maximum(adj_mx, adj_mx.transpose())#symmetric
print('num edge: ',np.sum(adj_mx)/2)
###node features
#e_mx
e_mx=np.zeros((num_node, num_node), dtype=np.float32)
e_mx[elements[:,0],elements[:,1]]=elements[:,2]
e_mx = np.maximum(e_mx, e_mx.transpose())#symmetric
#area_mx
area_mx=np.zeros((num_node, num_node), dtype=np.float32)
area_mx[elements[:,0],elements[:,1]]=elements[:,3]
area_mx = np.maximum(area_mx, area_mx.transpose())#symmetric
#dist_mx
dist_mx = np.zeros((num_node, num_node), dtype=np.float32)
dist_mx[:] = np.inf
for i in range(num_node):
    for j in range(num_node):
        x = float(node_loc[i][0]) - float(node_loc[j][0])
        y = float(node_loc[i][1]) - float(node_loc[j][1])
        z = float(node_loc[i][2]) - float(node_loc[j][2])
        x = math.pow(x,2)
        y = math.pow(y,2)
        z = math.pow(z,2)
        dis = math.sqrt(x+y+z)
        dist_mx[i, j] = dis
dist_mx=adj_mx*dist_mx
dist_mx = np.maximum(dist_mx, dist_mx.transpose())#symmetric
#virtual node
virtual_mx=np.zeros((num_node, num_node), dtype=np.float32)
virtual_mx[element_virtual[:,0],element_virtual[:,1]]=1
virtual_mx = np.maximum(virtual_mx, virtual_mx.transpose())#symmetric

# np.save('data/sensor_graph/adj_mx',adj_mx)
# np.save('data/sensor_graph/e_mx',e_mx)
# np.save('data/sensor_graph/area_mx',area_mx)
# np.save('data/sensor_graph/dist_mx',dist_mx)
# np.save('data/sensor_graph/virtual_mx',virtual_mx)

0 358
num node:  365
num edge:  486.0
